**Based on the previous base implementation of DPR, a little tweaked version for enabling the users to input question and the corresponding answer returned**

In [1]:
!pip install transformers torch torchvision torchaudio faiss-cpu numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 36.7 MB/s eta 0:00:00


In [2]:
from transformers import DPRQuestionEncoder, DPRContextEncoder, DPRQuestionEncoderTokenizer, DPRContextEncoderTokenizer
from transformers import BertForQuestionAnswering, BertTokenizer
import torch
import faiss
import numpy as np

In [3]:
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
context_encoder = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
context_tokenizer = DPRContextEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizer'.


### **Initialising BERT for the generation of text based on the context retrieved by DPR**

In [4]:
qa_model = BertForQuestionAnswering.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")
qa_tokenizer = BertTokenizer.from_pretrained("bert-large-uncased-whole-word-masking-finetuned-squad")

config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-large-uncased-whole-word-masking-finetuned-squad were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [5]:
contexts = [
    "Albert Einstein was a theoretical physicist known for his theory of relativity.",
    "Marie Curie was a physicist and chemist famous for her work on radioactivity.",
    "Isaac Newton formulated the laws of motion and universal gravitation."
]

In [7]:
context_inputs = context_tokenizer(contexts, padding=True, truncation=True, return_tensors="pt", max_length = 100)
context_embeddings = context_encoder(**context_inputs).pooler_output.detach().numpy()

In [8]:
index = faiss.IndexFlatL2(context_embeddings.shape[1])
index.add(np.array(context_embeddings))

In [9]:
def retrieve_best_context(question_embedding, index, contexts):
    distances, indices = index.search(np.array([question_embedding]), 1)
    return contexts[indices[0][0]]

In [10]:
def generate_answer_with_bert(question, context):
    inputs = qa_tokenizer(question, context, return_tensors="pt")
    with torch.no_grad():
        outputs = qa_model(**inputs)
        answer_start_index = torch.argmax(outputs.start_logits)
        answer_end_index = torch.argmax(outputs.end_logits) + 1
        answer = qa_tokenizer.convert_tokens_to_string(qa_tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start_index:answer_end_index]))
    return answer

**FLOW -> QUESTION -> QTOKENIZER -> QENCODER -> RETRIEVER -> RELATED CONTEXTS + QUESTION -> BERT**

###**Once retrieving the contexts, it is fed to the BERT model for the generation of answer.**

In [11]:
def answer_question(user_question):
    question_inputs = question_tokenizer(user_question, return_tensors="pt")
    question_embedding = question_encoder(**question_inputs).pooler_output.detach().numpy()[0]

    best_context = retrieve_best_context(question_embedding, index, contexts)
    answer = generate_answer_with_bert(user_question, best_context)
    return answer

In [13]:
while True:
    user_question = input("Ask a question (or type 'exit' to quit): ")
    if user_question.lower() == "exit":
        break
    answer = answer_question(user_question)
    print("Answer:", answer)

Ask a question (or type 'exit' to quit): who is known for theorey of relativity
Answer: albert einstein
Ask a question (or type 'exit' to quit): WHat is marie curie famous for?
Answer: her work on radioactivity
Ask a question (or type 'exit' to quit): who found 3 laws of motion?
Answer: isaac newton
Ask a question (or type 'exit' to quit): exit
